In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import scipy
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
import operator
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
import matplotlib
import seaborn as sns
from scipy import stats
import mpmath
from IPython.display import Image
import math

def metrics(O, P):
    mae = sklearn.metrics.mean_absolute_error(O, P)
    mdae = sklearn.metrics.median_absolute_error(O, P)
    RSq = sklearn.metrics.r2_score(O, P)
    rmse = sklearn.metrics.mean_squared_error(O, P, squared=False)
    R2 = stats.linregress(O, P)[2]**2
    return mae, mdae, rmse, RSq, R2

def plot_regression(O, P):
    plt.figure(figsize = (6, 6))
    plt.scatter(O, P, edgecolor='#808080', facecolors='#C0C0C0', linewidth=1, zorder = 2)
    corr, _ = pearsonr(O, P)
    regmodel = LinearRegression()
    regmodel.fit(O.values.reshape(-1,1), P)
    rsq = regmodel.score(O.values.reshape(-1,1), P)
    obs, pred = O.values.reshape(-1,1), regmodel.predict(O.values.reshape(-1,1))
    b, a = regmodel.intercept_, regmodel.coef_
    plt.plot([obs.min(), obs.max()], [pred.min(), pred.max()], color = 'black', lw = 3, zorder = 3)
    plt.plot([-5, obs.max()], [-5, obs.max()], color = 'navy', ls = '--', zorder = 1)
    plt.legend([u'Pearson R\u00b2: {:.3f}'.format(corr**2), '1:1 Plot', 'Observed - Predicted'], loc = 2)
    plt.xlabel('Observed')
    plt.ylabel('Predicted')
    plt.title("a&b = %.4f & %.4f" % (a, b))
    
def poly(x, y, deg):
    x = np.array(x)
    y = np.array(y)
    x = x[:, np.newaxis]
    y = y[:, np.newaxis]
    polynomial_features= PolynomialFeatures(degree=deg)
    x_poly = polynomial_features.fit_transform(x)
    model = LinearRegression()
    model.fit(x_poly, y)
    b, a = model.intercept_, model.coef_
    y_poly_pred = model.predict(x_poly)
    rmse = np.sqrt(mean_squared_error(y, y_poly_pred))
    r2 = r2_score(y,y_poly_pred)
    b = pd.DataFrame(b, columns = ['b'])
    a = pd.DataFrame(a)
    a = a.add_prefix('a').drop('a0', 1)
    fig = matplotlib.pyplot.gcf()
    fig.set_size_inches(6, 6)
    plt.grid(zorder = 1)
    plt.scatter(x, y, edgecolor='#808080', facecolors='#C0C0C0', linewidth=1, zorder = 2)
    sort_axis = operator.itemgetter(0)
    sorted_zip = sorted(zip(x,y_poly_pred), key=sort_axis)
    x, y_poly_pred = zip(*sorted_zip)
    plt.plot(x, y_poly_pred, color = 'navy', linestyle = '--', zorder = 3)
    plt.legend([u'Pearson R\u00b2: {:.3f} &\nPearson R: {:.3f}'.format(r2, r2 ** 0.5), '1:1 Plot'])
    plt.ylabel('Predicted')
    plt.xlabel('Observed')
    plt.grid()
    plt.show();
    return pd.concat([a, b], 1)

## Θεμελιώδη προβλήματα

Πρώτο θεμελιώδες

In [8]:
def fir_them(x1, y1, G, S):
    x2  = x1 + S * np.sin(G * np.pi / 200)
    y2 = y1 + S * np.cos(G * np.pi / 200)
    dx = x2 - x1
    dy = y2 - y1
    return (x2, y2, dx, dy)

Δεύτερο θεμελιώδες

In [9]:
def sec_them(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    G1 = np.arctan(abs(dx/dy)) * 200 / np.pi

    if dx > 0 and dy > 0:
        G = G1

    elif dx > 0 and dy < 0:
        G = 200 - G1

    elif dx < 0 and dy < 0:
        G = 200 + G1

    elif dx < 0 and dy > 0:
        G = 400 - G1
    
    elif dx == 0 and dy < 0:
        G = 200
    
    elif dx == 0 and dy > 0:
        G = 0
        
    elif dx < 0 and dy == 0:
        G = 300
        
    elif dx > 0 and dy == 0:
        G = 100

    elif dx == 0 and dy == 0:
        G = np.nan
        
    S = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return G, S

def third_them(b, G, n, truth = False):
    if truth:
        G2 = G + b + n * 200
    else:
        G2 = G + b + 200
    if G2 > 400:
        k = G2 // 400
        G2 = G2 - k * 400
    return G2

## Επίλυση Όδευσης

In [2]:
def dist(x1, y1, x2, y2):
    return(np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2))

def traverse(x1, y1, x2, y2, x3, y3, x4, y4, s, b, corr = True):
    GT1T2, sT1T2 = sec_them(x1, y1, x2, y2)
    GT3T4, sT3T4 = sec_them(x3, y3, x4, y4)

    dx_pr = x3 - x2
    dy_pr = y3 - y2

    S_sum = np.sum(s)
    b_sum = np.sum(b)

    wb = 5 * np.sqrt(n)
    Ds = 0.02 * np.sqrt(S_sum) + 0.2

    G_truth = third_them(b_sum, GT1T2, n, True)

    Wb = GT3T4 - G_truth
    if Wb < wb and corr:
        print('Angle Check:', np.round(abs((GT3T4 - G_truth) * 100), 2), '<', np.round(wb, 2))
    
    if corr:
        cor_angle = Wb / n
    else:
        cor_angle = 0

    G = np.zeros(len(b))
    G[0] = third_them((b[0]  + cor_angle), GT1T2, n)
    for i in range(1, len(G)):
        G[i] = np.round(third_them((b[i] + cor_angle), G[i - 1], n), 4)

    G = np.insert(G, 0, GT1T2)

    X0 = np.zeros(len(b))
    Y0 = np.zeros(len(b))
    X0[0], Y0[0] = fir_them(x1, y1, G[1], s[0])[2:]

    for i in range(0, len(G) - 1):
        X0[i], Y0[i] = fir_them(X0[i - 1], Y0[i - 1], G[i], s[i - 1])[2:]

    dx_truth = np.sum(X0[1:])
    dy_truth = np.sum(Y0[1:])
    dx = dx_pr - dx_truth
    dy = dy_pr - dy_truth

    ds = np.sqrt(dx ** 2 + dy ** 2)
    if ds < Ds and corr:
        print('Linear Check:', np.round(ds, 2), '<', np.round(Ds, 2))
    
    bwt_x = s/S_sum * dx
    bwt_y = s/S_sum * dy
    if not corr:
        bwt_x = np.zeros(len(bwt_x))
        bwt_y = np.zeros(len(bwt_y))

    X = np.zeros(len(b) - 1)
    Y = np.zeros(len(b) - 1)

    X[0] = x2
    Y[0] = y2

    for i in range(1, len(G) - 2):
        X[i] = X0[i] + bwt_x[i - 1] + X[i - 1]
        Y[i] = Y0[i] + bwt_y[i - 1] + Y[i - 1]

    X = np.insert(X, 0, x1)
    Y = np.insert(Y, 0, y1)
    X = np.insert(X, len(X), x3)
    Y = np.insert(Y, len(Y), y3)
    X = np.insert(X, len(X), x4)
    Y = np.insert(Y, len(Y), y4)

    stops = ['T1', 'T2']
    stops[2:] = list(np.arange(1, len(s)+1))
    stops[-1:] = ['T3', 'T4']

    coords = pd.DataFrame(data = [X, Y], index = ['X', 'Y'], columns = stops).T.round(decimals=3)

    X_ch = np.zeros(len(b))
    Y_ch = np.zeros(len(b))

    X_ch[0] = x2
    Y_ch[0] = y2

    for i in range(1, len(G) - 1):
        X_ch[i] = X0[i] + bwt_x[i - 1] + X_ch[i - 1]
        Y_ch[i] = Y0[i] + bwt_y[i - 1] + Y_ch[i - 1]

    X_ch = np.insert(X_ch, 0, x1)
    Y_ch = np.insert(Y_ch, 0, y1)
    X_ch = np.insert(X_ch, len(X_ch), x4)
    Y_ch = np.insert(Y_ch, len(Y_ch), y4)

    coords_check = pd.DataFrame(data = [X_ch, Y_ch], index = ['X', 'Y'], columns = stops).T
    return coords.round(3), coords_check.round(3)

## Αντίστροφη επίλυση όδευσης για εύρεση προβληματικής γωνία

In [ ]:
def check_angle_false(x1, y1, x2, y2, x3, y3, x4, y4, s, b):
    trav1 = traverse(x1, y1, x2, y2, x3, y3, x4, y4, s, b, corr = False)[1]
    x4, y4 = x1, y1
    x3, y3 = x2, y2
    x2, y2 = x3, y3
    x1, y1 = x4, y4

    stops = ['T1', 'T2']
    stops[2:] = list(np.arange(1, len(s)+1))
    stops[-1:] = ['T3', 'T4']
    b = [400 - x for x in b]
    b = list(reversed(b))
    s = list(reversed(s))
    stops_rev = list(reversed(stops))
    
    trav2 = traverse(x1, y1, x2, y2, x3, y3, x4, y4, s, b, corr = False)[1]
    trav2.index = stops_rev
    dd = trav1 - trav2
    false = np.sqrt(dd['X'] ** 2 + dd['Y'] ** 2).round(3)
    false = false.reindex(stops)
    false = pd.DataFrame(false, columns = ['Difference (m)'])
    return (false)

## Εμπροσθοτομία

In [18]:
url = 'https://lh3.googleusercontent.com/-hmTNO8lr4RE/VQLIylqRNNI/AAAAAAAAC94/0zKs9uIDSOI/w563-h338-no/%CE%95%CE%9C%CE%A0%CE%A1%CE%9F.jpg'

def apli_emprosthotomia(xa, ya, xb, yb, a, b):
    sfa = mpmath.cot(a / 200 * np.pi)
    sfb = mpmath.cot(b / 200 * np.pi)
    xm = float((yb - ya + xa * sfb + xb * sfa) / (sfa + sfb))
    ym = float((xa - xb + ya * sfb + yb * sfa) / (sfa + sfb))
    return xm, ym

Image(url= url)

## Πρόβλημα Hansen

In [17]:
url = 'https://upload.wikimedia.org/wikipedia/commons/f/fd/HansenPblm1.png'

def cot(g):
    return 1/math.tan(g*math.pi/200)

def hansen(points,gon):
    d1=gon[2]
    d2=gon[3]
    g1=gon[0]
    g2=gon[1]

    xa=points[0][0]
    ya=points[0][1]
    xb=points[1][0]
    yb=points[1][1]

    #print (g1,g2,d1,d2,xa,ya,xb,yb)
    d=d1+d2
    g=g1+g2
    nom=cot(d)-cot(g)+cot(g1)-cot(d2)
    denom=cot(g1)*cot(d2)-cot(g)*cot(d)
    lamda=200*math.atan(nom/denom)/math.pi
    b=200-(g+d2)
    omega=200-(b+g+lamda)
    a=200-d-g1
    theta=200-(a+d-lamda)
    A=theta
    B=b+omega
    nom1=yb-ya+xa*cot(B)+xb*cot(A)
    denom1=cot(A)+cot(B)
    nom2=xa-xb+ya*cot(B)+yb*cot(A)
    xm = nom1/denom1
    ym = nom2/denom1
    return xm, ym

Image(url= url)

In [13]:
# x1, y1 = 400402.39, 4541321.224
# x2, y2 = 400457.925, 4541331.536
# x3, y3 = 400481.859, 4541453.556
# x4, y4 = 400592.067, 4541528.300

# s = [36.574, 24.983, 19.71, 34.193, 30.382]

# b = [171.8085, 176.9944, 125.1023, 237.7836, 182.1094, 279.9456]

# n = len(b)

# x1, y1 = 281.73, 5818.96
# x2, y2 = 1248.42, 7143.24
# x3, y3 = 1866.37, 7124.23
# x4, y4 = 2332.15, 8626.04

# s = [112.43, 137.12, 124.19, 119.79, 127.16]

# b = [270.135, 187.428, 208.572, 189.114, 205.805, 118.018]

# n = len(b)

x1, y1 = 400661.16, 4540604.717
x2, y2 = 400726.724, 4540680.44
x3, y3 = 400726.724, 4540680.440
x4, y4 = 400661.160, 4540604.717

s = [31.373, 27.201, 18.392, 25.283, 46.098, 36.615, 31.548, 15.303, 23.384, 29.968]

b = [220.2089, 269.0663, 181.7531, 212.7550, 296.0945, 312.3851, 129.7549, 270.7580, 211.5708, 282.2047, 13.4706]

n = len(b)

traverse(x1, y1, x2, y2, x3, y3, x4, y4, s, b)[0]

Angle Check: 2.19 < 16.58
Linear Check: 0.08 < 0.54


,X,Y
T1,400661.160,4540604.717
T2,400726.724,4540680.440
1,400753.628,4540696.565
2,400776.880,4540682.463
3,400794.656,4540677.764
4,400817.317,4540666.569
5,400799.465,4540624.066
6,400769.068,4540644.496
7,400741.550,4540629.055
8,400728.915,4540637.695


In [12]:
check_angle_false(x1, y1, x2, y2, x3, y3, x4, y4, s, b)

,Difference (m)
T1,0.000
T2,124.346
1,73.191
2,48.377
3,14.939
4,68.833
T3,124.346
T4,280.816


In [5]:
a = 94.885
b = 56.093
xa, ya = 2500, 7000
xb, yb = 2515.31, 7124.49

apli_emprosthotomia(xa, ya, xb, yb, a, b)

(2638.8689386470687, 6994.1611391892975)